In [ ]:
#### IMPORTS ####
include("source/main.jl")

In [ ]:
manager(23, 0.1, collect(0:-0.06:-0.12), ["poff", "poff"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], figScale=6.)
# manager(37, 0.1, collect(0:-0.06:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], figScale=6.)
# manager(37, 0.1, collect(0:-0.04:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"])
# manager(43, 0.1, collect(0:-0.02:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], 2.)